# Dealing with Large Datasets in Pandas

In [1]:
!pip install faker --quiet

In [2]:
import pandas as pd
import numpy as np
import random
from faker import Faker
import datetime

In [ ]:
%%time

# Initialize Faker for generating fake data
fake = Faker()

# Set the number of rows
n_rows = 5_000_000  # You can change this to any number for a larger/smaller dataset

# Create the dataset
data = {
    "id": np.arange(1, n_rows + 1),  # Unique integer IDs
    "name": [fake.name() for _ in range(n_rows)],  # Random names
    "age": np.random.randint(18, 80, size=n_rows),  # Random ages between 18 and 80
    "email": [fake.email() for _ in range(n_rows)],  # Random email addresses
    "address": [fake.address().replace("\n", ", ") for _ in range(n_rows)],  # Random addresses
    "city": [fake.city() for _ in range(n_rows)],  # Random cities
    "country": [fake.country() for _ in range(n_rows)],  # Random countries
    "salary": np.round(np.random.uniform(30000, 150000, size=n_rows), 2),  # Random salaries between 30k and 150k
    "date_of_joining": [fake.date_between(start_date='-10y', end_date='today') for _ in range(n_rows)],  # Random dates
    "credit_score": np.random.randint(300, 850, size=n_rows),  # Random credit scores
}

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv("large_dataset.csv", index=False)

print("Dataset created and saved to 'large_dataset.csv'")


## Reading datasets efficiently

### Using `chunk_size` to read large datasets efficiently

In [ ]:
chunk_size = 50000  # Adjust chunk size as needed
for chunk in pd.read_csv("large_dataset.csv", chunksize=chunk_size):
    # Process each chunk
    print(chunk.head(1))


### Specify data types when loading

In [ ]:
dtypes = {
    'id': 'int32',
    'age': 'int8',
    'salary': 'float32',
    'credit_score': 'int16'
}
df = pd.read_csv('large_dataset.csv', dtype=dtypes)

### Use `nrows` to get data subset

In [ ]:
df = pd.read_csv('large_dataset.csv', nrows=100000)
print(len(df))

## Optimizing Memory Usage

### Downcasting numerical columns


In [ ]:
df['age'] = pd.to_numeric(df['age'], downcast='integer')
df['salary'] = pd.to_numeric(df['salary'], downcast='float')

### Convert categorical columns

In [ ]:
df['country'] = df['country'].astype('category')

### Using sparse data types

In [ ]:
df['credit_score'] = pd.Series(pd.arrays.SparseArray(df['credit_score']))

## Working with Chunks to perform operations

In [ ]:
chunk_size = 100000
total_salary = 0
row_count = 0

for chunk in pd.read_csv('large_dataset.csv', chunksize=chunk_size):
    total_salary += chunk['salary'].sum()
    row_count += len(chunk)

mean_salary = total_salary / row_count
print(f'Mean salary: {mean_salary}')


## Parallel Processing with Dask

In [ ]:
!pip install dask[dataframe] --quiet

In [ ]:
import dask.dataframe as dd

# Read the dataset using Dask
df = dd.read_csv('large_dataset.csv')

# Perform operations (e.g., calculate mean salary)
mean_salary = df['salary'].mean().compute()
print(f'Mean salary: {mean_salary}')

## Use `pyarrow` for Faster I/O

In [ ]:
!pip install pyarrow --quiet

## Save and read in parquet format


In [ ]:
df.to_parquet('large_dataset.parquet')  # Save as Parquet format
df = pd.read_parquet('large_dataset.parquet')  # Load Parquet file


## Vectorized Operations vs Loops

In [ ]:
%%time
df['new_col'] = df['age'] * 2

## Using SQL-like queries

In [ ]:
!pip install pandasql --quiet

In [ ]:
import psutil

# Get total memory
total_memory = psutil.virtual_memory().total

# Convert from bytes to gigabytes for easier reading
total_memory_gb = total_memory / (1024 ** 3)

print(f"Total RAM: {total_memory_gb:.2f} GB")

In [ ]:
import pandasql as psql

sql_like_query = """
SELECT TOP 100
  name,
  salary
FROM
  df
WHERE
  age > 30
ORDER BY
  salary DESC
"""
if total_memory_gb < 15:
    print('Not enough RAM to run this function. Please use High RAM environment')
else:
    result = psql.sqldf(sql_like_query, globals())